# DRUMS - COVASIM Data Generation

In [14]:
import sys
sys.path.append('../')
# from drums_data_gen import *
from drums_data_gen_multi import *
import Modules.Loaders.DataFormatter as DF
from Modules.Utils.Imports import *

In [15]:
def to_torch(ndarray):
    arr = torch.tensor(ndarray, dtype=torch.float)
    arr.requires_grad_(True)
    arr = arr.to(device)
    return arr

def to_numpy(x):
    return x.detach().cpu().numpy()

device = torch.device(GetLowestGPU(pick_from=[0,1,2,3]))

Device set to cpu


- `drums_gata_gen` generates data for one simulation.
- `drums_data_gen_multi` generates data for multiple simulations and averages over the results.

***

- `test_prob`: This is a float value that corresponds to the probability of testing given the agent is symptomatic. Currently, the values for proabbility of testing for other scenarios are `asymp_prob=0.001`, `symp_quar_prob=0.3`, and `asymp_quar_prob=0.3`. Additionally, the policy for quarantining is `quar_policy='daily'`
- `trace_lb` and `trace_ub`: These float values correspond to the lower and upper bounds on probability of tracing. Currently, the trace probability for the layers are `h=1.0`, `s=0.5`, `w=0.5`, `c=0.3`.
- `chi_type`: The type of the function that interacts with the tracing probability. Can choose between `constant`, `linear`, `piecewise`, and `sin`.
- `dynamic`: Boolean value that indicates whether or not to include the interacting term/function.
- `keep_d`: Boolean value that indicates wehter or not to include the D - diagnosed compartments.
- `model_params:` Dictionary containing keys pointing to parameter values, the compartment model's values over the simulation, and more.

In [16]:
population = int(50e3)
test_prob = 0.1
trace_lb = 0.0
trace_ub = 0.3
chi_type = 'piecewise'
keep_d = True
dynamic = True
maskb = True
multiple = True
n_runs = 1000
model_params = ModelParams(population, test_prob, trace_lb, trace_ub, chi_type, keep_d, dynamic, maskb)

In [17]:
# drums_data_generator(model_params)
drums_data_generator_multi(model_params, n_runs)

Initializing sim with 50000 people for 182 days
  Running "Sim 0": 2020-02-01 ( 0/182) (0.43 s)  ———————————————————— 1%
  Running "Sim 0": 2020-02-11 (10/182) (0.64 s)  •——————————————————— 6%
  Running "Sim 0": 2020-02-21 (20/182) (0.90 s)  ••—————————————————— 11%
  Running "Sim 0": 2020-03-02 (30/182) (1.14 s)  •••————————————————— 17%
  Running "Sim 0": 2020-03-12 (40/182) (1.42 s)  ••••———————————————— 22%
  Running "Sim 0": 2020-03-22 (50/182) (1.74 s)  •••••——————————————— 28%
  Running "Sim 0": 2020-04-01 (60/182) (2.02 s)  ••••••—————————————— 33%
  Running "Sim 0": 2020-04-11 (70/182) (2.34 s)  •••••••————————————— 39%
  Running "Sim 0": 2020-04-21 (80/182) (2.68 s)  ••••••••———————————— 44%
  Running "Sim 0": 2020-05-01 (90/182) (2.99 s)  •••••••••——————————— 50%
  Running "Sim 0": 2020-05-11 (100/182) (3.32 s)  •••••••••••————————— 55%
  Running "Sim 0": 2020-05-21 (110/182) (3.61 s)  ••••••••••••———————— 61%
  Running "Sim 0": 2020-05-31 (120/182) (3.89 s)  •••••••••••••—

In [ ]:
path = '../Data/covasim_data/drums_data/'
retrain = False
case_name = get_case_name(population, test_prob, trace_ub, keep_d, dynamic=dynamic, chi_type=chi_type)

if maskb:
    case_name = case_name + '_masking'

if multiple:
    case_name = case_name + '_' + str(n_runs)
    params = DF.load_covasim_data(path, population, test_prob, trace_ub, keep_d, case_name, plot=True)
else:
    params = DF.load_covasim_data(path, population, test_prob, trace_ub, keep_d, case_name, plot=True)

In [ ]:
if multiple:
    data = np.mean(params['data'], axis=0)
    data = (data / params['population'])
else:
    data = params['data']
    data = (data / params['population']).to_numpy()

params.pop('data')
N = len(data)
t = np.arange(N)[:,None]

In [ ]:
# rescale data values to represent number of people in state X rather than ratios
data *= population

plot=True
if plot:
    n = data.shape[1]
    col_names = list('STEAYDQRF') if keep_d else list('STEAYQRF')
    # plot compartments
    fig = plt.figure(figsize=(15, 15))
    
    if multiple:
        for i in range(1, n + 1):
            ax = fig.add_subplot(int(np.ceil(n / 3)), 3, i)
            ax.plot(t, data[:, i - 1], '.k', label='Covasim Data')
            ax.set_title(col_names[i - 1])
            ax.legend(fontsize=8)
            fig.subplots_adjust(left=0, right=1, bottom=0, top=1)
            plt.tight_layout(pad=2)
            plt.savefig(os.path.join(path, case_name + '_avg' + '.png') )
            # plt.show()
    else:
        for i in range(1, n + 1):
            ax = fig.add_subplot(int(np.ceil(n / 3)), 3, i)
            ax.plot(t, data[:, i - 1], '.k', label='Covasim Data')
            ax.set_title(col_names[i - 1])
            ax.legend(fontsize=8)
            fig.subplots_adjust(left=0, right=1, bottom=0, top=1)
            plt.tight_layout(pad=2)
            plt.savefig(os.path.join(path, case_name + '_single' + '.png') )
            # plt.show()